In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir("/content/drive/MyDrive/fraud-detect-drive/code-files/initial_code_analysis_POC_12.08.2023")
!ls
import pandas as pd
import numpy as np

analysis_temp.ipynb
apps_and_reviews_jon.xlsx
chunked_files
combined_reviews_given_data.csv
ml_model_for_classification.ipynb
mrinal_xgboost_lendingapps.ipynb
own_apps_classifcation_data_2.csv
own_apps_classifcation_data.csv
own_apps_review_data.csv
personalloan_app_list_extended_2021_labelled.xlsx
reviews_classification_labels.gdoc
suspect_personalloan_apps_NG_IN_reviews_data_2021_labelled.xlsx
suspect_personalloan_apps_PH_US_reviews_data_2021_labelled.xlsx


In [3]:
from  IPython. display  import  clear_output
import warnings
warnings. filterwarnings('ignore')


# installs
!pip install datasets
!pip install transformers==4.28.0
!pip install sentencepiece==0.1.98
!pip install pysentimiento
!pip install faker

In [4]:
#load the given app_data from jon
df_app_class = pd.read_csv("own_apps_classifcation_data.csv")
display(df_app_class.columns)
display(df_app_class.info())

Index(['package_name', 'title', 'short_desc', 'market_url', 'type_postreview',
       'labels'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5106 entries, 0 to 5105
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   package_name     5106 non-null   object
 1   title            5106 non-null   object
 2   short_desc       5024 non-null   object
 3   market_url       5106 non-null   object
 4   type_postreview  5106 non-null   object
 5   labels           5106 non-null   object
dtypes: object(6)
memory usage: 239.5+ KB


None

In [5]:
#load the given app_data from jon
df = pd.read_excel("personalloan_app_list_extended_2021_labelled.xlsx",sheet_name="Sheet1")
display(df.columns)
display(df.shape)

Index(['dataset', '_merge_reviewdata', 'id', 'random_cohort', 'random',
       'package_name', 'edate_created', 'title', 'short_desc', 'market_url',
       ...
       'fake_reviews.2', 'developer_comment_responses',
       'developer_comment_responses_susp', '_merge_suspectapplist', 'today',
       'dayssincecreation', 'dayssincelastupdate', 'reviews_to_ratings',
       'reviews_to_downloads', 'hasreviews_42matters'],
      dtype='object', length=144)

(5106, 144)

In [6]:
print("\b total columns:\b",len(df.columns))
print("------------------------------------")
for x in df.columns: print(x)

 total columns: 144
------------------------------------
dataset
_merge_reviewdata
id
random_cohort
random
package_name
edate_created
title
short_desc
market_url
type_postreview
country_flag
fake_reviews
comments
Unnamed: 14
ir_local_allowed
annualized_ir
APR
fees_local_allowed
fees
lending_app
type
fake_reviews.1
selected1
selected2
description
developer
email
website
physical_address
price_numeric
version
contains_ads
from_developer
size
main_language
screenshots_count
version_code
edate_file
cat_key
edate_lastupdated
num_countries_appavailable
num_languages
appavailable_US
appavailable_onlyUS
appavailable_IN
appavailable_onlyIN
appavailable_PH
appavailable_onlyPH
appavailable_NG
appavailable_onlyNG
downloads
downloads_min
downloads_max
number_ratings
mean_rating
ratings_1
ratings_2
ratings_3
ratings_4
ratings_5
iap
iap_min
iap_max
market_status
num_similarapps
downloads_estimated
ratings_estdownloads
stores_itunes
interelmt_digitalpurchases
interelmt_unrestrictedinternet
interelmt

code used to get the combined reviews

```

#load the reviews data that is required
df_review = pd.read_csv("/content/drive/MyDrive/fraud-detect-drive/code-files/initial_code_analysis_POC_12.08.2023/chunked_files/reviews_000_00.txt", sep= '\t')

from tqdm import tqdm

folder_path = "/content/drive/MyDrive/fraud-detect-drive/code-files/initial_code_analysis_POC_12.08.2023/chunked_files"  # Replace with the actual folder path
filtered_df = pd.DataFrame()

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".txt"):  # Change the extension as needed
        file_path = os.path.join(folder_path, filename)
        df_review = pd.read_csv(file_path, sep='\t',header=None,
                                names=["package_name", "body", "lang", "review_date", "rating"])  # Adjust delimiter as needed
        
        # Filter the new DataFrame based on "package_name"
        filtered_new_df = df_review[df_review["package_name"].isin(df["package_name"])]
        
        # Append to the existing filtered DataFrame
        filtered_df = filtered_df.append(filtered_new_df, ignore_index=True)


combined_reviews = []

for package_name in filtered_df["package_name"].unique():
    package_reviews = filtered_df[filtered_df["package_name"] == package_name]["body"]
    combined_review = "\n".join(str(review) for review in package_reviews)  # Join individual reviews
    combined_reviews.append({"package_name": package_name, "combined_review": combined_review})

# Convert the list of dictionaries to a DataFrame
combined_reviews_df = pd.DataFrame(combined_reviews)
combined_reviews_df

combined_reviews_df.to_csv("combined_reviews_given_data.csv",index=False)
combined_reviews_new_df = pd.read_csv("combined_reviews_given_data.csv")
combined_reviews_new_df

```



# code for machine_learning model for reviews classification


```
#load the reviews and classification data
df_collected_reviews = pd.read_csv("own_apps_review_data.csv")
df_collected_reviews = df_collected_reviews[["review_body","labels"]].convert_dtypes()
df_collected_reviews = df_collected_reviews.dropna()


import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import torch

# Assuming you have your train_df, val_df, and test_df loaded
train_df, temp_df = train_test_split(df_collected_reviews, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

MAX_LENGTH = 128  # Max sequence length

def convert_to_dataset(df):
    df = {"text": df['review_body'].tolist(), "label": df["labels"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

def convert_to_dataset_test(df):
    df = {"text": df['review_body'].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")  # Change to desired tokenizer

# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset_test(test_df)

# Tokenize and prepare datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"],
                                   "attention_mask": train_encodings["attention_mask"],
                                   "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"],
                                 "attention_mask": val_encodings["attention_mask"],
                                 "label": val_dataset["label"]})

test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"],
                                  "attention_mask": test_encodings["attention_mask"]})

# Define compute_f1_score function
def compute_f1_score(pred):
    # pred is a tuple (predictions, labels)
    predictions, labels = pred
    # Compute the F1 score
    f1 = f1_score(labels, predictions.argmax(axis=1), average='macro')
    return {"f1_score": f1}


# Load model
num_labels = 3  # Number of unique labels in your dataset
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=num_labels)


# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Training arguments
training_args = TrainingArguments(
    output_dir="./classification_results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    evaluation_strategy="epoch",
    logging_dir="./logs",
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_f1_score  # Make sure you have the compute_f1_score function defined
)

# Train the model
trainer.train()

----
Output:
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 [8322/8322 22:08, Epoch 3/3]
Epoch	Training Loss	Validation Loss	F1 Score
1	0.294600	0.226546	0.611659
2	0.225900	0.245098	0.761027
3	0.149500	0.255098	0.682266
-------

Scores on test set
# Get predictions on the validation set
val_predictions = trainer.predict(val_dataset)
val_pred_labels = np.argmax(val_predictions.predictions, axis=1)
val_true_labels = val_dataset["label"] # Convert to list for to_device

val_true_labels_device = torch.tensor(val_true_labels, device=device)
val_pred_labels_device = torch.tensor(val_pred_labels, device=device)

val_accuracy = accuracy_score(val_true_labels, val_pred_labels)
val_f1_score = f1_score(val_true_labels, val_pred_labels, average='macro')  # Change to 'micro' if needed

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

--------
Output
Validation Accuracy: 0.9149089597980891
Validation F1 Score: 0.6822658663329341

```



here is the code used to make percentage assumptions for the apps
```
from collections import Counter

def calculate_percentage(values):
    value_counts = Counter(values)
    total_values = len(values)
    
    unique_values = [0, 1, 2]  # List of unique values you're interested in
    value_counts_list = [value_counts[value] for value in unique_values]
    percentages_list = [(count / total_values) * 100 for count in value_counts_list]
    
    return percentages_list

    from tqdm import tqdm

local_list1 = []
for items in tqdm(combined_reviews_new_df["combined_review"]):
  if items is not None:
      items = items.split("\n")
      # Tokenize the input strings
      item_encodings = tokenizer(items, padding=True, truncation=True, return_tensors="pt", max_length=128)

      # Move the input tensors to the GPU if available
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
      input_ids = item_encodings["input_ids"].to(device)
      attention_mask = item_encodings["attention_mask"].to(device)

      # Create a dictionary in the format expected by the Trainer
      items_dict = {"input_ids": input_ids, "attention_mask": attention_mask}
      item_dataset = Dataset.from_dict(items_dict)

      with torch.no_grad():
          predictions = trainer.predict(item_dataset).predictions
          probabilities = np.exp(predictions - np.max(predictions, axis=1, keepdims=True))
          probabilities /= np.sum(probabilities, axis=1, keepdims=True)
          preds = np.argmax(probabilities, axis = 1)
          local_list1.append(calculate_percentage(preds))
  else:
      local_list1.append([0.0,0.0,0.0])


combined_reviews_new_df["review_percentages"] = pd.Series(local_list1)
combined_reviews_new_df.to_csv("combined_reviews_given_data.csv",index=False)
```



# analysis_dummy

In [7]:
#load the combined reviews from all the chunks jon provided
combined_reviews_new_df = pd.read_csv("combined_reviews_given_data.csv")
combined_reviews_new_df.head()

,package_name,combined_review,review_percentages
0,adria.com.awb.egypt.entreprises,Perfect\nGreat Application 👍 and easy use 😉\nA...,"[79.48717948717949, 5.128205128205128, 15.3846..."
1,ai.deepscore.acom.prod,Legit\nMy application is in processing I hope ...,"[88.58447488584474, 10.045662100456621, 1.3698..."
2,aib.ibank.android,"A terrible app. Full of bugs, always asking to...","[92.31193672981424, 5.352216295751334, 2.33584..."
3,am.imwallet.android,My Favorite App Thank you\nVery good implement...,"[85.78947368421052, 13.368421052631579, 0.8421..."
4,anitech.atmbalancecheack,Bank balance check app looks very interesting ...,"[35.0, 48.5, 16.5]"


In [8]:
print(combined_reviews_new_df.shape)
print(combined_reviews_new_df.info())

(1020, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   package_name        1020 non-null   object
 1   combined_review     1020 non-null   object
 2   review_percentages  1020 non-null   object
dtypes: object(3)
memory usage: 24.0+ KB
None


In [9]:
# create a list of our conditions
conditions = [
    (df['type_postreview'] == "legitimate"),
    (df['type_postreview'] == "removed, legitimate"),
    (df['type_postreview'] == "predatory"),
    (df['type_postreview'] == "fraud"),
    (df['type_postreview'] == "removed, fraud"),
    (df['type_postreview'] == "removed")
    ]

# create a list of the values we want to assign for each condition
values = [0, 0, 1, 1, 1, 1]

# create a new column and use np.select to assign values to it using our lists as arguments
df['label'] = np.select(conditions, values)

# drop the string column
df= df.drop('type_postreview', axis=1)

In [10]:
#create the dataframe for final analysis

#these values are taken from Mrinal's XG Boost code, we can add more fields as needed
final_data_df = df[['dataset', 'label', 'package_name', 'short_desc','lending_app', 'num_countries_appavailable',
       'num_languages', 'downloads_min', 'downloads_max', 'number_ratings',
       'mean_rating', 'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4',
       'ratings_5', 'contains_ads', 'size', 'screenshots_count',
       'version_code', 'ratings_estdownloads', 'stores_itunes',
       'interelmt_digitalpurchases', 'interelmt_unrestrictedinternet',
       'interelmt_sharesinfo', 'interelmt_shareslocation',
       'interelmt_inapppurchase', 'interelmt_usersinteract',
       'anyphysical_address', 'specific_physical_address', 'anywebsite',
       'website_appads', 'anyemail', 'gmailemail', 'genericemail',
       'anyprivacy_policy', 'anypromo_video', 'anyprice',
       'developer_has_other_apps', 'developer_num_other_apps', 'anysdks',
       'num_sdks', 'num_permissions']]

final_data_df = final_data_df.merge(combined_reviews_new_df, on="package_name", how= "outer").drop(columns=["combined_review"])
final_data_df["review_percentages"] = final_data_df["review_percentages"].apply(lambda x: [0.0, 0.0, 0.0] if pd.isna(x) else x)

In [11]:
print(final_data_df.shape)
print(len(final_data_df.package_name.unique()))
print(len(final_data_df.short_desc))
print(final_data_df["review_percentages"].value_counts())

(5106, 44)
5106
5106
[0.0, 0.0, 0.0]                                                4086
[100.0, 0.0, 0.0]                                                89
[50.0, 0.0, 50.0]                                                 6
[66.66666666666666, 0.0, 33.33333333333333]                       4
[66.66666666666666, 33.33333333333333, 0.0]                       3
                                                               ... 
[86.71023965141612, 4.11522633744856, 9.174534011135318]          1
[15.35448491970231, 82.84371327849588, 1.8018018018018018]        1
[87.03956343792633, 8.458390177353342, 4.502046384720328]         1
[83.46839546191248, 2.2690437601296596, 14.26256077795786]        1
[93.33333333333333, 3.3333333333333335, 3.3333333333333335]       1
Name: review_percentages, Length: 908, dtype: int64


In [12]:
final_data_df = final_data_df.convert_dtypes()
null_counts = final_data_df.isna().sum()
print(null_counts)

dataset                              0
label                                0
package_name                         0
short_desc                          82
lending_app                         18
num_countries_appavailable           0
num_languages                        0
downloads_min                       52
downloads_max                        0
number_ratings                       0
mean_rating                       1530
ratings_1                         1558
ratings_2                         1558
ratings_3                         1558
ratings_4                         1558
ratings_5                         1558
contains_ads                         0
size                                24
screenshots_count                    0
version_code                      1058
ratings_estdownloads                52
stores_itunes                        0
interelmt_digitalpurchases           0
interelmt_unrestrictedinternet       0
interelmt_sharesinfo                 0
interelmt_shareslocation 

In [13]:
#replace all the null values
final_data_df["specific_physical_address"] = final_data_df["specific_physical_address"].fillna(0)
final_data_df["short_desc"] = final_data_df["short_desc"].fillna("None")
final_data_df["ratings_estdownloads"] = final_data_df["ratings_estdownloads"].fillna(0)
final_data_df["version_code"] = final_data_df["version_code"].fillna(00000)
final_data_df["lending_app"] = final_data_df["lending_app"].fillna(0)
final_data_df["ratings_1"] = final_data_df["ratings_1"].fillna(0)
final_data_df["ratings_2"] = final_data_df["ratings_2"].fillna(0)
final_data_df["ratings_3"] = final_data_df["ratings_3"].fillna(0)
final_data_df["ratings_4"] = final_data_df["ratings_4"].fillna(0)
final_data_df["ratings_5"] = final_data_df["ratings_5"].fillna(0)
final_data_df["mean_rating"] = final_data_df["mean_rating"].fillna(0)
final_data_df["downloads_min"] = final_data_df["downloads_min"].fillna(0)
final_data_df["size"] = final_data_df["size"].fillna(0)

In [14]:
null_counts = final_data_df.isna().sum()
print(null_counts)

dataset                           0
label                             0
package_name                      0
short_desc                        0
lending_app                       0
num_countries_appavailable        0
num_languages                     0
downloads_min                     0
downloads_max                     0
number_ratings                    0
mean_rating                       0
ratings_1                         0
ratings_2                         0
ratings_3                         0
ratings_4                         0
ratings_5                         0
contains_ads                      0
size                              0
screenshots_count                 0
version_code                      0
ratings_estdownloads              0
stores_itunes                     0
interelmt_digitalpurchases        0
interelmt_unrestrictedinternet    0
interelmt_sharesinfo              0
interelmt_shareslocation          0
interelmt_inapppurchase           0
interelmt_usersinteract     

In [15]:
final_data_df.dataset.value_counts()

train_validation    4596
test                 510
Name: dataset, dtype: Int64

In [16]:
final_data_df.dtypes

dataset                            string
label                               Int64
package_name                       string
short_desc                         string
lending_app                         Int64
num_countries_appavailable          Int64
num_languages                       Int64
downloads_min                       Int64
downloads_max                       Int64
number_ratings                      Int64
mean_rating                       Float64
ratings_1                           Int64
ratings_2                           Int64
ratings_3                           Int64
ratings_4                           Int64
ratings_5                           Int64
contains_ads                        Int64
size                                Int64
screenshots_count                   Int64
version_code                        Int64
ratings_estdownloads              Float64
stores_itunes                       Int64
interelmt_digitalpurchases          Int64
interelmt_unrestrictedinternet    

In [17]:
float_columns = [
    'lending_app', 'num_countries_appavailable', 'num_languages', 'downloads_min',
    'downloads_max', 'number_ratings', 'mean_rating', 'ratings_1', 'ratings_2',
    'ratings_3', 'ratings_4', 'ratings_5', 'contains_ads', 'size', 'screenshots_count',
    'version_code', 'ratings_estdownloads', 'stores_itunes', 'interelmt_digitalpurchases',
    'interelmt_unrestrictedinternet', 'interelmt_sharesinfo', 'interelmt_shareslocation',
    'interelmt_inapppurchase', 'interelmt_usersinteract', 'anyphysical_address',
    'specific_physical_address', 'anywebsite', 'website_appads', 'anyemail',
    'gmailemail', 'genericemail', 'anyprivacy_policy', 'anypromo_video',
    'anyprice', 'developer_has_other_apps', 'developer_num_other_apps',
    'anysdks', 'num_sdks', 'num_permissions'
]

# Convert specified columns to float data type
for column in float_columns:
    try:
      final_data_df[column] = final_data_df[column].astype(float)
    except:
      print(column)

In [18]:
#create datasets from the final data to train the model <-- this is not used
train_df = final_data_df[final_data_df["dataset"]=="train_validation"].drop(columns= ["dataset"])
print("trian_data_shape:",train_df.shape)
test_df = final_data_df[final_data_df["dataset"]=="test"].drop(columns= ["dataset"])
print("test_data_shape:",test_df.shape)

trian_data_shape: (4596, 43)
test_data_shape: (510, 43)


Model to classify reviews as fraud or not. using XLM-Roberta for clasification.

In [19]:
#this function created dataloader for our data <- you can modify this for other implementations
import ast
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader, random_split

class CustomDataset(Dataset):
    def __init__(self, df, transformer_model, max_seq_length):
        self.data = df
        self.tokenizer = AutoTokenizer.from_pretrained(transformer_model)
        self.transformer = AutoModel.from_pretrained(transformer_model)
        self.max_seq_length = max_seq_length

    def convert_to_list(self, input_value):
        if isinstance(input_value, str):
            try:
                return ast.literal_eval(input_value)
            except (SyntaxError, ValueError):
                return input_value
        else:
            return input_value

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # Encode the text input using the transformer tokenizer
        encoded_short_desc = self.tokenizer(str(row['short_desc']), padding='max_length', truncation=True, max_length=self.max_seq_length, return_tensors='pt')
        input_ids_short_desc = encoded_short_desc['input_ids'].squeeze()
        attention_mask_short_desc = encoded_short_desc['attention_mask'].squeeze()

        # Convert review_percentages string to a list of floats
        review_percentages = self.convert_to_list(row['review_percentages'])

        # Extract other non-text features
        non_text_features = row.drop(['package_name', 'short_desc', 'review_percentages',"label"]).values
        non_text_features = non_text_features.astype(float)  # Convert to float data type
        non_text_features = torch.tensor(non_text_features,dtype=torch.float32)
        label = torch.tensor(row["label"], dtype=torch.long)
        pkg_name = row['package_name']

        # Assuming non_text_features is a NumPy array
        # for column_idx in range(non_text_features.shape[1]):
        #     column_data = non_text_features[:, column_idx]
        #     column_name = f"Column_{column_idx}"  # Replace with actual column name if available
        #     print(f"Column: {column_name}, Data Type: {column_data.dtype}")


        # Append review_percentages to non_text_features
        review_percentages_tensor = torch.tensor(review_percentages)
        non_text_features = torch.cat((non_text_features, review_percentages_tensor))

        return {
            'package_name': pkg_name,
            'input_ids_short_desc': input_ids_short_desc,
            'attention_mask_short_desc': attention_mask_short_desc,
            'non_text_features': non_text_features,
            'label': label
        }

In [20]:
#create a general dataset and then pass it into dataloader
all_data_df = final_data_df.drop(columns= ["dataset"])
print("all_data_df:",all_data_df.shape)

all_data_df: (5106, 43)


In [38]:
transformer_model = 'bert-base-cased'

In [39]:
transformer_model = 'xlm-roberta-base'

In [47]:
transformer_model = "ProsusAI/finbert"

In [40]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoModel

# Define the NonTextModel
class NonTextModel(nn.Module):
    def __init__(self, input_dim,non_text_expand):
        super(NonTextModel, self).__init__()
        self.fc = nn.Linear(input_dim, non_text_expand)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc(x)
        x = self.relu(x)
        # print("x:-------------------------------", x.shape)
        return x

# Define the TextClassificationHead
class TextClassificationHead(nn.Module):
    def __init__(self, bert_model, text_expand):
        super(TextClassificationHead, self).__init__()
        self.bert_model = bert_model
        self.fc = nn.Linear(self.bert_model.config.hidden_size, text_expand)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert_model(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.fc(pooled_output)
        # print("logits------------------------", logits.shape)
        return logits

# Define the CombinedModel
class CombinedModel(nn.Module):
    def __init__(self, text_model, non_text_model, num_classes, text_expand):
        super(CombinedModel, self).__init__()
        self.text_model = text_model
        self.non_text_model = non_text_model
        self.final_classifier = nn.Linear(text_expand + non_text_expand, num_classes)

    def forward(self, text_input_ids, text_attention_mask, non_text_inputs):
        text_features = self.text_model(text_input_ids, text_attention_mask)
        non_text_features = self.non_text_model(non_text_inputs)
        combined_features = torch.cat((text_features, non_text_features), dim=1)
        # print("combined_features:", combined_features.shape)
        logits = self.final_classifier(combined_features)
        return logits

# Create instances of NonTextModel and TextClassificationHead
non_text_input_dim = 42
non_text_expand = 64
non_text_model = NonTextModel(non_text_input_dim,non_text_expand)

bert_model = AutoModel.from_pretrained(transformer_model)
num_classes = 3
text_expand = 64
text_model = TextClassificationHead(bert_model, text_expand)

# Load your data and create a DataLoader
data = all_data_df
max_seq_length = 128
custom_dataset = CustomDataset(train_df, transformer_model, max_seq_length)

# Create DataLoader
batch_size = 16
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [48]:
# Create an instance of CombinedModel
model = CombinedModel(text_model, non_text_model, num_classes, text_expand)

# Move the combined model to the desired device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

CombinedModel(
  (text_model): TextClassificationHead(
    (bert_model): XLMRobertaModel(
      (embeddings): XLMRobertaEmbeddings(
        (word_embeddings): Embedding(250002, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): XLMRobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x XLMRobertaLayer(
            (attention): XLMRobertaAttention(
              (self): XLMRobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): XLMRobertaSelfOutput(
     

In [36]:
#results with bert base cased

import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

# Define hyperparameters
num_classes = 3
num_epochs = 10
learning_rate = 1e-5


# Create k-fold cross-validation splits
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store evaluation metrics for each fold
accuracy_scores = []
f1_scores = []

# Iterate over k-fold splits
for fold, (train_idx, val_idx) in enumerate(kf.split(data_loader.dataset.data, data_loader.dataset.data['label'])):
    print(f"Fold {fold + 1}")

    # Create train and validation data loaders for this fold
    train_data = torch.utils.data.Subset(data_loader.dataset, train_idx)
    val_data = torch.utils.data.Subset(data_loader.dataset, val_idx)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size)

    # Initialize model, optimizer, and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Training loop for this fold
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for batch in train_loader:
            # Forward pass
            text_input_ids = batch['input_ids_short_desc'].to(device)
            text_attention_mask = batch['attention_mask_short_desc'].to(device)
            non_text_inputs = batch['non_text_features'].to(device)
            labels = batch['label'].to(device)

            logits = model(text_input_ids, text_attention_mask, non_text_inputs)
            loss = criterion(logits, labels)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Validation
        model.eval()
        val_predictions = []
        val_labels = []
        with torch.no_grad():
            for batch in val_loader:
                text_input_ids = batch['input_ids_short_desc'].to(device)
                text_attention_mask = batch['attention_mask_short_desc'].to(device)
                non_text_inputs = batch['non_text_features'].to(device)
                labels = batch['label'].to(device)

                logits = model(text_input_ids, text_attention_mask, non_text_inputs)
                _, predicted = torch.max(logits, dim=1)
                # print("predicted", predicted)

                val_predictions.extend(predicted.tolist())
                val_labels.extend(labels.tolist())

        val_accuracy = accuracy_score(val_labels, val_predictions)
        val_f1 = f1_score(val_labels, val_predictions, average='weighted')

        print(f"Epoch {epoch + 1} - Loss: {total_loss:.4f} - Val Accuracy: {val_accuracy:.4f} - Val F1: {val_f1:.4f}")

    accuracy_scores.append(val_accuracy)
    f1_scores.append(val_f1)

# Print average evaluation metrics across all folds
print("Average Accuracy:", np.mean(accuracy_scores))
print("Average F1 Score:", np.mean(f1_scores))


Fold 1
Epoch 1 - Loss: 16705362.4015 - Val Accuracy: 0.7152 - Val F1: 0.6992
Epoch 2 - Loss: 13889863.7594 - Val Accuracy: 0.7250 - Val F1: 0.7027
Epoch 3 - Loss: 11212611.8129 - Val Accuracy: 0.6826 - Val F1: 0.6866
Epoch 4 - Loss: 8336193.7178 - Val Accuracy: 0.7098 - Val F1: 0.6977
Epoch 5 - Loss: 5639997.8034 - Val Accuracy: 0.6870 - Val F1: 0.6885
Epoch 6 - Loss: 3671983.7886 - Val Accuracy: 0.6489 - Val F1: 0.6638
Epoch 7 - Loss: 1877249.8996 - Val Accuracy: 0.7011 - Val F1: 0.7016
Epoch 8 - Loss: 972846.8975 - Val Accuracy: 0.7543 - Val F1: 0.6959
Epoch 9 - Loss: 590793.5821 - Val Accuracy: 0.7511 - Val F1: 0.6902
Epoch 10 - Loss: 561533.8718 - Val Accuracy: 0.6413 - Val F1: 0.6582
Fold 2
Epoch 1 - Loss: 534325.4346 - Val Accuracy: 0.7650 - Val F1: 0.7275
Epoch 2 - Loss: 518351.0423 - Val Accuracy: 0.7193 - Val F1: 0.7103
Epoch 3 - Loss: 532531.0123 - Val Accuracy: 0.7443 - Val F1: 0.6794
Epoch 4 - Loss: 446153.1939 - Val Accuracy: 0.6942 - Val F1: 0.7001
Epoch 5 - Loss: 513840.

In [43]:
#results with xlm-roberta
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

# Define hyperparameters
num_classes = 3
num_epochs = 5
learning_rate = 1e-5


# Create k-fold cross-validation splits
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store evaluation metrics for each fold
accuracy_scores = []
f1_scores = []

# Iterate over k-fold splits
for fold, (train_idx, val_idx) in enumerate(kf.split(data_loader.dataset.data, data_loader.dataset.data['label'])):
    print(f"Fold {fold + 1}")

    # Create train and validation data loaders for this fold
    train_data = torch.utils.data.Subset(data_loader.dataset, train_idx)
    val_data = torch.utils.data.Subset(data_loader.dataset, val_idx)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size)

    # Initialize model, optimizer, and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Training loop for this fold
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for batch in train_loader:
            # Forward pass
            text_input_ids = batch['input_ids_short_desc'].to(device)
            text_attention_mask = batch['attention_mask_short_desc'].to(device)
            non_text_inputs = batch['non_text_features'].to(device)
            labels = batch['label'].to(device)

            logits = model(text_input_ids, text_attention_mask, non_text_inputs)
            loss = criterion(logits, labels)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Validation
        model.eval()
        val_predictions = []
        val_labels = []
        with torch.no_grad():
            for batch in val_loader:
                text_input_ids = batch['input_ids_short_desc'].to(device)
                text_attention_mask = batch['attention_mask_short_desc'].to(device)
                non_text_inputs = batch['non_text_features'].to(device)
                labels = batch['label'].to(device)

                logits = model(text_input_ids, text_attention_mask, non_text_inputs)
                _, predicted = torch.max(logits, dim=1)
                # print("predicted", predicted)

                val_predictions.extend(predicted.tolist())
                val_labels.extend(labels.tolist())

        val_accuracy = accuracy_score(val_labels, val_predictions)
        val_f1 = f1_score(val_labels, val_predictions, average='weighted')

        print(f"Epoch {epoch + 1} - Loss: {total_loss:.4f} - Val Accuracy: {val_accuracy:.4f} - Val F1: {val_f1:.4f}")

    accuracy_scores.append(val_accuracy)
    f1_scores.append(val_f1)

# Print average evaluation metrics across all folds
print("Average Accuracy:", np.mean(accuracy_scores))
print("Average F1 Score:", np.mean(f1_scores))

Fold 1
Epoch 1 - Loss: 41743217.5251 - Val Accuracy: 0.6815 - Val F1: 0.6432
Epoch 2 - Loss: 34819125.5502 - Val Accuracy: 0.6826 - Val F1: 0.6449
Epoch 3 - Loss: 29366737.1491 - Val Accuracy: 0.7011 - Val F1: 0.6540
Epoch 4 - Loss: 24137731.6264 - Val Accuracy: 0.3424 - Val F1: 0.3204
Epoch 5 - Loss: 17995089.9383 - Val Accuracy: 0.3783 - Val F1: 0.3868
Fold 2
Epoch 1 - Loss: 12250825.6305 - Val Accuracy: 0.4548 - Val F1: 0.4801
Epoch 2 - Loss: 6672496.7257 - Val Accuracy: 0.4483 - Val F1: 0.4730
Epoch 3 - Loss: 2591579.4809 - Val Accuracy: 0.7160 - Val F1: 0.6888
Epoch 4 - Loss: 2476602.5735 - Val Accuracy: 0.3112 - Val F1: 0.2756
Epoch 5 - Loss: 2302035.7518 - Val Accuracy: 0.7051 - Val F1: 0.6864
Fold 3


KeyboardInterrupt: ignored

In [49]:
#results with finbert

import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

# Define hyperparameters
num_classes = 3
num_epochs = 5
learning_rate = 1e-5


# Create k-fold cross-validation splits
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store evaluation metrics for each fold
accuracy_scores = []
f1_scores = []

# Iterate over k-fold splits
for fold, (train_idx, val_idx) in enumerate(kf.split(data_loader.dataset.data, data_loader.dataset.data['label'])):
    print(f"Fold {fold + 1}")

    # Create train and validation data loaders for this fold
    train_data = torch.utils.data.Subset(data_loader.dataset, train_idx)
    val_data = torch.utils.data.Subset(data_loader.dataset, val_idx)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size)

    # Initialize model, optimizer, and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Training loop for this fold
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for batch in train_loader:
            # Forward pass
            text_input_ids = batch['input_ids_short_desc'].to(device)
            text_attention_mask = batch['attention_mask_short_desc'].to(device)
            non_text_inputs = batch['non_text_features'].to(device)
            labels = batch['label'].to(device)

            logits = model(text_input_ids, text_attention_mask, non_text_inputs)
            loss = criterion(logits, labels)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Validation
        model.eval()
        val_predictions = []
        val_labels = []
        with torch.no_grad():
            for batch in val_loader:
                text_input_ids = batch['input_ids_short_desc'].to(device)
                text_attention_mask = batch['attention_mask_short_desc'].to(device)
                non_text_inputs = batch['non_text_features'].to(device)
                labels = batch['label'].to(device)

                logits = model(text_input_ids, text_attention_mask, non_text_inputs)
                _, predicted = torch.max(logits, dim=1)
                # print("predicted", predicted)

                val_predictions.extend(predicted.tolist())
                val_labels.extend(labels.tolist())

        val_accuracy = accuracy_score(val_labels, val_predictions)
        val_f1 = f1_score(val_labels, val_predictions, average='weighted')

        print(f"Epoch {epoch + 1} - Loss: {total_loss:.4f} - Val Accuracy: {val_accuracy:.4f} - Val F1: {val_f1:.4f}")

    accuracy_scores.append(val_accuracy)
    f1_scores.append(val_f1)

# Print average evaluation metrics across all folds
print("Average Accuracy:", np.mean(accuracy_scores))
print("Average F1 Score:", np.mean(f1_scores))

Fold 1
Epoch 1 - Loss: 49145522.6904 - Val Accuracy: 0.7207 - Val F1: 0.6134
Epoch 2 - Loss: 39777862.9956 - Val Accuracy: 0.7217 - Val F1: 0.6159
Epoch 3 - Loss: 32882009.8639 - Val Accuracy: 0.7207 - Val F1: 0.6142
Epoch 4 - Loss: 26036223.1196 - Val Accuracy: 0.7196 - Val F1: 0.6188
Epoch 5 - Loss: 19675554.1431 - Val Accuracy: 0.7196 - Val F1: 0.6227
Fold 2
Epoch 1 - Loss: 13267812.4547 - Val Accuracy: 0.7182 - Val F1: 0.6135
Epoch 2 - Loss: 7096222.1140 - Val Accuracy: 0.7193 - Val F1: 0.6229
Epoch 3 - Loss: 2129443.5415 - Val Accuracy: 0.4472 - Val F1: 0.4779
Epoch 4 - Loss: 1284448.3698 - Val Accuracy: 0.7334 - Val F1: 0.6362
Epoch 5 - Loss: 1078451.9546 - Val Accuracy: 0.5832 - Val F1: 0.5965
Fold 3
Epoch 1 - Loss: 866926.3647 - Val Accuracy: 0.7149 - Val F1: 0.6700
Epoch 2 - Loss: 586068.5739 - Val Accuracy: 0.4723 - Val F1: 0.5009
Epoch 3 - Loss: 449636.2354 - Val Accuracy: 0.3036 - Val F1: 0.2584
Epoch 4 - Loss: 393500.5651 - Val Accuracy: 0.7127 - Val F1: 0.6494
Epoch 5 - L

# dummy code

In [29]:
df.columns

Index(['dataset', '_merge_reviewdata', 'id', 'random_cohort', 'random',
       'package_name', 'edate_created', 'title', 'short_desc', 'market_url',
       ...
       'fake_reviews.2', 'developer_comment_responses',
       'developer_comment_responses_susp', '_merge_suspectapplist', 'today',
       'dayssincecreation', 'dayssincelastupdate', 'reviews_to_ratings',
       'reviews_to_downloads', 'hasreviews_42matters'],
      dtype='object', length=144)

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from xgboost import XGBClassifier

class CustomModel(nn.Module):
    def __init__(self, transformer_model, transformer_tokenizer):
        super(CustomModel, self).__init__()

        # Pretrained transformer blocks
        self.transformer_reviews = transformer_model
        self.transformer_description = transformer_model
        self.xgboost_model = XGBClassifier()  # Initialize XGBClassifier

        self.tokenizer = transformer_tokenizer

        # FCN layers
        self.fcn = nn.Sequential(
            nn.Linear(in_features=1546, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=128),
            nn.ReLU()
        )

        # Classification layer
        self.classifier = nn.Linear(in_features=128, out_features=num_classes)

    def forward(self, reviews, description, non_text_inputs):
        # Encode the reviews using the tokenizer and pass through the transformer
        encoded_reviews = self.tokenizer(reviews, padding=True, truncation=True, return_tensors="pt", max_length=max_seq_length, return_attention_mask=True)
        input_ids_reviews = encoded_reviews["input_ids"].to(device)
        attention_mask_reviews = encoded_reviews["attention_mask"].to(device)
        reviews_embedding = self.transformer_reviews(input_ids=input_ids_reviews, attention_mask=attention_mask_reviews)['last_hidden_state'][:, 0, :]

        # Encode the description using the tokenizer and pass through the transformer
        encoded_description = self.tokenizer(description, padding=True, truncation=True, return_tensors="pt", max_length=description_length, return_attention_mask=True)
        input_ids_description = encoded_description["input_ids"].to(device)
        attention_mask_description = encoded_description["attention_mask"].to(device)
        description_embedding = self.transformer_description(input_ids=input_ids_description, attention_mask=attention_mask_description)['last_hidden_state'][:, 0, :]

        # Pass non-text inputs through XGBoost model
        non_text_embedding = non_text_inputs

        # Concatenate the embeddings
        combined_embedding = torch.cat([reviews_embedding, description_embedding, non_text_embedding], dim=1)
        print(combined_embedding.shape)
        # Pass through the FCN layers
        fcn_output = self.fcn(combined_embedding)

        # Pass through the classifier
        logits = self.classifier(fcn_output)

        return logits

# Example usage
transformer_model = AutoModel.from_pretrained('bert-base-uncased')
transformer_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

num_classes = 2
batch_size = 16
max_seq_length = 128
description_length = 128
num_non_text_inputs = 10

model = CustomModel(transformer_model, transformer_tokenizer)

# Move the model to the desired device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([16, 1546])


In [ ]:
# Generate some example inputs
from faker import Faker
fake = Faker()
reviews = [fake.sentence() for _ in range(batch_size)]
description = [fake.sentence() for _ in range(batch_size)]
non_text_inputs = torch.randn((batch_size, num_non_text_inputs), device=device)

# Forward pass
logits = model(reviews, description, non_text_inputs)

#Next Steps
1. Finetune the model using Optuna.
2. Publish the results

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from transformers import AutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

#Define the dataset

# Split the dataset into train and test sets
X_text_train, X_text_test, X_description_train, X_description_test, X_non_text_train, X_non_text_test, y_train, y_test = train_test_split(
    X_text, X_description, X_non_text, y, test_size=0.2, random_state=42
)

# Define the hyperparameters
num_classes = 10
batch_size = 16
max_seq_length = 128
description_length = 128
num_non_text_inputs = 10
num_epochs = 10
learning_rate = 1e-3

# Create the dataset and data loaders
train_dataset = MyDataset(X_text_train, X_description_train, X_non_text_train, y_train)
test_dataset = MyDataset(X_text_test, X_description_test, X_non_text_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Initialize the models
transformer_model = AutoModel.from_pretrained('bert-base-uncased')
transformer_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# xgboost_model = XGBClassifier()
model = CustomModel(transformer_model, transformer_tokenizer, xgboost_model)

# Move the model to the desired device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
total_steps = len(train_loader)
for epoch in range(num_epochs):
    model.train()
    for i, (reviews, description, non_text_inputs, labels) in enumerate(train_loader):
        reviews = reviews.to(device)
        description = description.to(device)
        non_text_inputs = non_text_inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        logits = model(reviews, description, non_text_inputs)
        loss = criterion(logits, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_steps}], Loss: {loss.item():.4f}")

    # Evaluate on the test set
    model.eval()
    with torch.no_grad():
        all_predictions = []
        all_labels = []
        for reviews, description, non_text_inputs, labels in test_loader:
            reviews = reviews.to(device)
            description = description.to(device)
            non_text_inputs = non_text_inputs.to(device)
            labels = labels.to(device)

            logits = model(reviews, description, non_text_inputs)
            predictions = torch.argmax(logits, dim=1)

            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_predictions)
        print(f"Test Accuracy: {accuracy:.4f}")

torch.Size([16, 10])
